<a href="https://colab.research.google.com/github/radhakrishnan-omotec/arwan-iris-dog-repo/blob/main/ISEF_FINAL_ArwanMakhija_Prediction_Resnet152_CNN_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# Enhanced Python Notebook for **TailSense** : ResNet152-based Canine Pet Image and Audio Spectrogram Classification

### Author : ARWAN MAKHIJA

Below is an enhanced Python notebook implementation for Google Colab that integrates both image classification and spectrogram audio classification using the ResNet152 model, optimized for maximum accuracy and depth.

It leverages ResNet152’s deep architecture (~60M parameters) with residual connections for classifying dog emotions from both image and audio data derived from videos of a Cocker Spaniel.

The dataset is assumed to contain 8-10 emotion classes (e.g., "defensive," "stressed," "friendly"), and the implementation includes data preprocessing, model training, evaluation, a Gradio interface for real-time inference, and TensorFlow Lite conversion for edge deployment.

# ResNet152-based Image Classification

# **Part 1** : Cocker Spaniel 8 Emotions Prediction Resnet152 CNN Model Traning


---

# My Project Key Enhancements:
**8-Class Cocker Spaniel Emotion Classifier:**<br>
Replaced the 3-class tongue classification (Healthy Tongue, Tongue with Moderate Coating, Tongue with Severe Coating) with an 8-class emotion classification (Sad, Happy, Stress, Restless, Normal, Love, Unhappy, Tired).
Updated dataset paths to a hypothetical Cocker Spaniel emotion dataset, with class folders named after emotions.<br><br>
Modified class labels, image counting, and visualization to handle 8 classes, ensuring compatibility with the new dataset structure.<br><br>
Adjusted the confusion matrix and classification report to reflect the increased number of classes, with a larger figure size for clarity.<br><br><br>

**ResNet152-Based Transfer-Learning Model:**<br>
Replaced the Sequential CNN with a pre-trained ResNet152 model from tensorflow.keras.applications, using ImageNet weights for transfer learning.
Froze the base model layers (base_model.trainable = False) to leverage pre-trained features and reduce training time.<br><br>
Added custom layers: GlobalAveragePooling2D for spatial dimension reduction, two dense layers (512 and 256 units) with ReLU activation, and dropout layers (0.5 and 0.3) to prevent overfitting. The final dense layer outputs probabilities for 8 classes with softmax activation.<br><br>
Updated the Grad-CAM function to use the last convolutional layer of ResNet152 (conv5_block3_3_conv), ensuring compatibility with the new model architecture.<br>

---

**ISEF Reviewer Notes:**<br>
The code assumes the existence of a Cocker Spaniel emotion dataset at /content/drive/MyDrive/Cocker_Spaniel_Emotions_Dataset.zip, with subfolders named after each emotion (Sad, Happy, etc.). Users must provide this dataset and update paths accordingly.<br><br>
The Grad-CAM implementation uses the conv5_block3_3_conv layer, which is the last convolutional layer in ResNet152. Users should verify this using model.summary() if the architecture differs.<br><br>
The notebook is designed for Google Colab with GPU acceleration, as specified in the original metadata.<br><br>
The code supports .jpg and .png formats, accommodating common image types in emotion datasets.<br><br>
Training epochs are set to 50, with EarlyStopping to prevent overfitting; users may adjust based on dataset size and convergence behavior.<br>

---
---

#Cocker_Spaniel_Emotion_ResNet152

### Enhanced for 8-class Cocker Spaniel emotion prediction using ResNet152 with transfer learning.

# 1) Import necessary libraries

In [ ]:
# Import necessary libraries
from google.colab import drive
import tensorflow as tf
import os
import shutil
import random
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
import cv2
import PIL

# 1) Mount Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# 1) Check TensorFlow version

In [ ]:
# Check TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")

# 1) Unzip the dataset (assumed to be a Cocker Spaniel emotion dataset)

In [ ]:
# Unzip the dataset (assumed to be a Cocker Spaniel emotion dataset)
!unzip "/content/drive/MyDrive/Cocker_Spaniel_Emotions_Dataset.zip" -d "/content/drive/MyDrive/Cocker_Spaniel_Emotions_Dataset"

# 1) Define dataset directory

In [ ]:
# Define dataset directory
dataset_dir = "/content/drive/MyDrive/Cocker_Spaniel_Emotions_Dataset/Cocker_Spaniel_Emotions_Dataset"
dataset_dir = pathlib.Path(dataset_dir)

# 1) Count total images

In [ ]:
# Count total images
total_images = len(list(dataset_dir.glob('*/*.jpg'))) + len(list(dataset_dir.glob('*/*.png')))
print(f"Total images in dataset: {total_images}")

# 1) Define emotion classes and labels

In [ ]:
# Define emotion classes and labels
emotion_classes = ["Sad", "Happy", "Stress", "Restless", "Normal", "Love", "Unhappy", "Tired"]
class_labels = {cls: idx for idx, cls in enumerate(emotion_classes)}
image_counts = {}
image_paths = {}

# 1) Count images per class

In [ ]:
# Count images per class
for cls in emotion_classes:
    count = len(list(dataset_dir.glob(f'{cls}/*')))
    paths = list(dataset_dir.glob(f'{cls}/*'))
    image_counts[cls] = count
    image_paths[cls] = paths
    print(f"{cls}: {count} images")

# 1) Visualize sample images

In [ ]:
# Visualize sample images
for cls, paths in image_paths.items():
    if paths:
        print(f"\nSample {cls} Image:")
        PIL.Image.open(str(paths[min(10, len(paths)-1)])).show()

# 1) Plot class distribution

In [ ]:
# Plot class distribution
plt.figure(figsize=(10, 6))
plt.bar(image_counts.keys(), image_counts.values(), color='skyblue')
plt.xlabel('Emotion Class')
plt.ylabel('Number of Images')
plt.title('Distribution of Cocker Spaniel Emotion Images')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 1) Split dataset into train, validation, and test sets

In [ ]:
# Split dataset into train, validation, and test sets
def split_dataset(dataset_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_dir, split), exist_ok=True)

    for cls in emotion_classes:
        class_path = os.path.join(dataset_dir, cls)
        images = os.listdir(class_path)
        random.shuffle(images)

        total_images = len(images)
        train_end = int(train_ratio * total_images)
        val_end = train_end + int(val_ratio * total_images)

        train_images = images[:train_end]
        val_images = images[train_end:val_end]
        test_images = images[val_end:]

        def copy_images(image_list, split):
            split_class_dir = os.path.join(output_dir, split, cls)
            os.makedirs(split_class_dir, exist_ok=True)
            for img in image_list:
                src = os.path.join(class_path, img)
                dst = os.path.join(split_class_dir, img)
                shutil.copy(src, dst)

        copy_images(train_images, 'train')
        copy_images(val_images, 'val')
        copy_images(test_images, 'test')

splitted_dataset_dir = '/content/drive/MyDrive/Cocker_Spaniel_Emotions_Splitted'
split_dataset(dataset_dir, splitted_dataset_dir)
print("✅ Dataset successfully split into training, validation, and testing sets!")

# 1) Verify split counts

In [ ]:
# Verify split counts
for split in ['train', 'val', 'test']:
    print(f"\n📂 {split.upper()} SET:")
    split_path = os.path.join(splitted_dataset_dir, split)
    for cls in emotion_classes:
        class_path = os.path.join(split_path, cls)
        num_images = len(os.listdir(class_path)) if os.path.exists(class_path) else 0
        print(f"   - {cls}: {num_images} images")

# 1) Data augmentation and generators

In [ ]:
# Data augmentation and generators
IMG_HEIGHT, IMG_WIDTH, BATCH_SIZE = 224, 224, 32
NUM_CLASSES = 8

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    f'{splitted_dataset_dir}/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    f'{splitted_dataset_dir}/val',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    f'{splitted_dataset_dir}/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print("\nClass indices:", train_generator.class_indices)

# 1) Define ResNet152-based model

In [ ]:
# Define ResNet152-based model
def create_resnet152_model():
    base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model.trainable = False  # Freeze base model layers

    inputs = base_model.input
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

model = create_resnet152_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# 1) Define callbacks

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Cocker_Spaniel_Emotions/emotion_model_best.h5',
                            monitor='val_accuracy', save_best_only=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)


# 1) Train the model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, checkpoint, lr_scheduler]
)

# 1) Evaluate on test set

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\n✅ Test Accuracy: {test_accuracy * 100:.2f}%")

# 1) Plot training and validation metrics

In [ ]:
# Plot training and validation metrics
def plot_training_metrics(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy', marker='o')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy', marker='x')
    plt.title('📈 Training & Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss', marker='o', linestyle='--')
    plt.plot(epochs_range, val_loss, label='Validation Loss', marker='x', linestyle='--')
    plt.title('📉 Training & Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_training_metrics(history)

# 1) Confusion matrix and classification report

In [ ]:
# Confusion matrix and classification report
def plot_confusion_matrix(model, test_generator):
    class_names = list(test_generator.class_indices.keys())
    y_pred_probs = model.predict(test_generator)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = test_generator.classes

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix 📊')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.xticks(rotation=45, ha='right')
    plt.show()

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

plot_confusion_matrix(model, test_generator)

# 1) Grad-CAM for interpretability

In [ ]:
# Grad-CAM for interpretability
def get_gradcam_heatmap(model, img_array, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        class_output = predictions[:, predicted_class]

    grads = tape.gradient(class_output, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(conv_outputs, pooled_grads), axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

def display_gradcam(img_path, model, last_conv_layer_name):
    img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    heatmap = get_gradcam_heatmap(model, img_array, last_conv_layer_name)
    heatmap = cv2.resize(heatmap, (IMG_WIDTH, IMG_HEIGHT))

    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * 0.4 + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title('Grad-CAM Heatmap')
    plt.axis('off')

    plt.show()

# 1) Grad-CAM visualization

In [ ]:
# Grad-CAM visualization
img_path = '/content/drive/MyDrive/Cocker_Spaniel_Emotions_Splitted/val/Happy/sample_image.jpg'  # Update with actual path
display_gradcam(img_path, model, 'conv5_block3_3_conv')  # Last conv layer in ResNet152

# 1) Save the Resnet152 model

In [ ]:
# Save the Resnet152 model
model.save('/content/drive/MyDrive/Cocker_Spaniel_Emotions/emotion_detection_resnet152.h5')
print("✅ Model saved successfully as 'emotion_detection_resnet152.h5'")

# 1) Test prediction
## Test prediction on a single image

In [ ]:
# Test prediction on a single image
def predict_single_image(img_path, model):
    img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)
    predicted_class = emotion_classes[np.argmax(prediction)]

    plt.imshow(img)
    plt.title(f"Predicted Emotion: {predicted_class}")
    plt.axis('off')
    plt.show()

predict_single_image(img_path, model)

---
---
---
---
